# Downloading the OpenAQ Air Pollution dataset

In [1]:
# Uncomment below to download the Open AQ dataset from Amazon
#!pip install awscli
#!~/.local/bin/aws s3 cp --no-sign-request s3://openaq-data-archive/records/csv.gz/ dataset --recursive

# Now that the download is over, let's do some Air Pollution Data Exploration!

In [1]:
# Importing libraries
import numpy as np 
import pandas as pd
from pyspark.sql import SparkSession

In [2]:
# Creating a spark session
spark = SparkSession.builder.appName('232 Project').getOrCreate()

In [34]:
# Create a folder to put the unzipped csv files
#!mkdir -p combined_csv_files

In [37]:
# This will find the .csv.gz files and unzip them then copy that to the folder
#!find ./data/ -type f -name "*.csv" -exec gunzip -c {} \; -exec cp {} combined_csv_files/ \;

In [1]:
#!find ./combined_csv_files/ -type f -name "*.csv.gz" -exec gunzip -c {} \;

In [7]:
import os

# Print the current working directory
current_directory = os.getcwd()
print("Current Directory:", current_directory)

Current Directory: /expanse/lustre/projects/uci150/ameek1


In [ ]:
path = "/expanse/lustre/projects/uci150/cmerry/data/records/csv.gz"  

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType, DecimalType

schema = StructType([
    StructField("location_id", IntegerType(), True),
    StructField("sensors_id", IntegerType(), True),
    StructField("location", StringType(), True),
    StructField("datetime", TimestampType(), True),
    StructField("lat", DecimalType(precision=10, scale=6), True),
    StructField("lon", DecimalType(precision=10, scale=6), True),
    StructField("parameter", StringType(), True),
    StructField("units", StringType(), True),
    StructField("value", DecimalType(precision=10, scale=6), True),
])


# Read CSV files matching the pattern into a DataFrame
#df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(path)
df = spark.read.option("recursiveFileLookup", "true").csv(path, header=True, schema=schema)

In [11]:
df.show()

AttributeError: 'DataFrameReader' object has no attribute 'show'

## Transforming the datetime col ie splitting the 'datetime' column into date and time and then dropping the 'datetime' column

In [ ]:
from pyspark.sql.functions import unix_timestamp, from_unixtime, date_format, col
df = df.withColumn('ut', unix_timestamp(col('datetime'), 'yyyy-MM-dd HH:mm:ss')) \
       .withColumn('dty', from_unixtime('ut')) \
       .withColumn('date', date_format('dty', 'yyyy-MM-dd')) \
       .withColumn('time', date_format('dty', 'HH:mm:ss'))
df.show()

In [ ]:
df.columns

In [ ]:
df.printSchema()

In [ ]:
# Are there any null values? 
from pyspark.sql.functions import isnan, when, count, col

df.select(*(count(when(col(c).isNull(), c)).alias(c) for c in df.columns)).show()

In [ ]:
# Seeing if there are any rows where the value of the sensor measurement is below zero 
import pyspark.sql.functions as func

df.filter(func.col("value") <= 0).count()

In [ ]:
# Filtering for sensor 1803 and parameter 'pm10'
filtered_df = df.filter((df.sensors_id == 1803) & (df.parameter == "pm10")).orderBy("datetime")

In [ ]:
# selecting datetime and value for the graph
pm10_data = filtered_df.select("datetime", "value").collect()

In [ ]:
import matplotlib.pyplot as plt

# Extracting separate lists of dates and values
dates = [data.datetime for data in pm10_data]
values = [data.value for data in pm10_data]

# Plotting
plt.figure(figsize=(10, 5))
plt.plot(dates, values, marker='o', linestyle='-', color='b')
plt.title("PM10 Levels Over Time for Sensor 1803 at Location 1000")
plt.xlabel("Datetime")
plt.ylabel("PM10 Value (µg/m³)")
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()  
plt.show()

In [ ]:
df.createOrReplaceTempView('pollution')

In [ ]:
# Looking at the different number of parameters in the dataset
results = spark.sql("SELECT count(distinct parameter) as Parameter_Count \
                    FROM pollution")
results.show()

In [ ]:
# Looking at the sensor counts for each location. 
results = spark.sql("SELECT location_id, sensors_id, count(sensors_id) as Sensor_Count \
                    FROM pollution \
                    GROUP BY location_id, sensors_id \
                    ORDER BY location_id")
results.show()

# WHO air quality guidelines recommend a maximum exposure of 20 µg/m3 for PM10, 35 µg/m3 for PM2.5, and 0.070 parts per million for O3

## Find the data in the maximum polluted thresholds

In [ ]:
## Find the data in the polluted thresholds
results = spark.sql("SELECT * \
                    FROM pollution \
                    WHERE (parameter = 'pm10' AND value > 20) OR (parameter = 'pm25' AND value > 12) OR (parameter = 'o3' AND value > 0.07)")
results.show()
# Could gather insights about what the most polluted areas are 

### As we can see this occurs the same day around the same time period - ie. morning to midday 

## Average Value per measurement for each year for each location

In [ ]:
results = spark.sql("SELECT location_id, sensors_id, date, parameter, AVG(value) AS avg_value\
                    FROM pollution\
                    GROUP BY parameter, sensors_id, date, location_id")
results.show()